# 📥 Notebook 01 – Téléchargement des Spectres DR5

Ce notebook télécharge un lot de spectres .fits.gz du catalogue LAMOST DR5 à l’aide du script `dr5_downloader.py`.  
Il est conçu pour être exécuté **seulement lorsque de nouveaux spectres sont nécessaires**, afin d’alimenter le répertoire `data/raw/` en données brutes.

## ⚙️ Paramètres principaux

- `limit` : Nombre de plans de spectres à télécharger (chaque plan contient plusieurs spectres).
- `max_spectres` : Nombre maximum de spectres à télécharger.

## 🔗 Dépendances

- Le script Python `../src/tools/dr5_downloader.py` doit être présent et opérationnel.
- La configuration du projet doit pointer vers le bon répertoire `data/raw/`.

<br/>

**Exécution** : Lance la cellule ci-dessous pour télécharger de nouveaux spectres depuis le catalogue public DR5.


In [ ]:
import sys
import subprocess
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import os
from datetime import datetime, timezone
import re

# --- Widgets pour l'interface ---
limit_widget = widgets.IntText(value=5, description='Plans :')
max_spectra_widget = widgets.IntText(value=100, description='Spectres :')
run_button = widgets.Button(description="Lancer le téléchargement", button_style='success', icon='download')

# --- Zone d'affichage dynamique ---
# Un widget HTML pour la barre de progression qui se met à jour sur place
progress_display = widgets.HTML(value="")
# Une zone de sortie pour les messages de log normaux
log_output = widgets.Output()

# Afficher l'interface utilisateur
display(widgets.HBox([limit_widget, max_spectra_widget]), run_button, progress_display, log_output)

# --- Fonction de callback pour le bouton ---
def on_run_clicked(b):
    # Nettoyer les zones de sortie pour un nouveau run
    progress_display.value = ""
    with log_output:
        clear_output()
        
    # --- Préparation de la commande ---
    limit = str(limit_widget.value)
    max_spectra = str(max_spectra_widget.value)
    python_exe = sys.executable
    script_path = "../src/tools/dr5_downloader.py"
    cmd = [python_exe, script_path, "--limit", limit, "--max-spectres", max_spectra]
    
    progress_display.value = f"<b>Lancement du téléchargement...</b>"
    
    # --- Exécution et Capture en Temps Réel ---
    output_lines = []
    try:
        process = subprocess.Popen(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            bufsize=1,
            universal_newlines=True,
            encoding='utf-8',
            errors='replace'
        )

        if process.stdout:
            for line in iter(process.stdout.readline, ''):
                output_lines.append(line)
                
                # --- Logique d'affichage dynamique ---
                if 'Téléchargement:' in line and '%' in line:
                    # C'est une ligne de tqdm, on la traite pour l'affichage HTML
                    clean_line = line.strip()
                    match = re.search(r'(\d+)%\|', clean_line)
                    percent = int(match.group(1)) if match else 0
                    
                    progress_bar_html = f"""
                    <div style="font-family: monospace; white-space: pre;">{clean_line}</div>
                    <progress value="{percent}" max="100" style="width: 100%; height: 20px;"></progress>
                    """
                    progress_display.value = progress_bar_html
                else:
                    # C'est une autre ligne de log, on l'affiche dans la zone de sortie
                    with log_output:
                        print(line, end='')
        
        process.wait()
        full_output = "".join(output_lines)

        # --- Journalisation ---
        timestamp = datetime.now(timezone.utc).strftime("%Y%m%dT%H%M%SZ")
        log_dir = "../logs"
        os.makedirs(log_dir, exist_ok=True)

        if process.returncode == 0:
            progress_display.value = "<b>Téléchargement terminé avec succès.</b>"
            log_filename = f"download_log_{timestamp}.txt"
            log_path = os.path.join(log_dir, log_filename)
            with open(log_path, "w", encoding="utf-8") as f:
                f.write(full_output)
            with log_output:
                print(f"\nLog de succès sauvegardé dans : {log_path}")
        else:
            raise subprocess.CalledProcessError(process.returncode, cmd, output=full_output)

    except subprocess.CalledProcessError as e:
        progress_display.value = f"<b>Une erreur est survenue pendant le téléchargement (code {e.returncode}).</b>"
        with log_output:
             print(e.output or "Aucune sortie d'erreur capturée.")
        # ... (logique de sauvegarde du log d'erreur) ...
    except Exception as e:
        progress_display.value = f"<b>Une erreur inattendue est survenue : {e}</b>"

# --- Liaison ---
run_button.on_click(on_run_clicked)

Button(button_style='success', description='Lancer le téléchargement', icon='download', style=ButtonStyle())

HTML(value='')

Output()

#

# 🧹 Option de Nettoyage
Il peut arriver que tu souhaites nettoyer entièrement le répertoire ``data/raw/`` avant de relancer un téléchargement, notamment pour repartir d’un dossier vierge ou libérer de l’espace.

Cette option de nettoyage intègre deux niveaux de protection pour éviter toute suppression accidentelle :

- Confirmation interactive :
Un bouton de confirmation via un widget (ou une autre méthode comme input()) demande explicitement ton accord avant d’exécuter la suppression.

- Sauvegarde automatique :
Avant de vider le répertoire, le script réalise une copie complète de data/raw/ dans un dossier temporaire nommé data/raw_backup/.

Cela te permet de restaurer tes spectres si jamais tu as validé la suppression par erreur.

**Important :**

- Le dossier de sauvegarde est écrasé à chaque nouvelle exécution de l’option.

- Tu peux restaurer des fichiers en récupérant les spectres depuis data/raw_backup/ tant que tu n’as pas relancé un nettoyage.

**Pourquoi utiliser cette option ?**

- Repartir sur un lot de spectres neuf sans mélange avec d’anciens fichiers.
- Tester un workflow complet à partir d’un état initial propre.
- Gagner en traçabilité et éviter les erreurs lors de sessions consécutives de téléchargement.

In [ ]:
import ipywidgets as widgets
import shutil, os
from IPython.display import display

confirm = widgets.ToggleButtons(
    options=["Non", "Oui"],
    description='Confirmer suppr. ?',
    disabled=False
)
display(confirm)

def on_change(change):
    if change['new'] == "Oui":
        backup_dir = "../data/raw_backup/"
        try:
            shutil.copytree("../data/raw/", backup_dir, dirs_exist_ok=True)
            shutil.rmtree("../data/raw/")
            os.makedirs("../data/raw/")
            print("data/raw vidé et sauvegardé dans raw_backup/")
        except Exception as e:
            print(f"Erreur : {e}")

confirm.observe(on_change, names='value')


#

## Tableau de Bord de l'État du Dataset

1. **Importe ``DatasetBuilder`` :** On réutilise l'outil conçu pour gérer les lots d'entraînement.
2. ``_list_available_fits()`` : On appelle cette méthode pour qu'elle scanne le dossier ``data/raw/`` et compte tous les fichiers ``.fits.gz`` téléchargés.
3. ``_load_trained_log()`` : On appelle cette méthode pour qu'elle lise le ``fichier trained_spectra.csv`` et compte combien de spectres ont déjà été marqués comme "_utilisés_".
4. **Calcul :** Une simple soustraction nous donne le nombre de spectres qui sont nouveaux et prêts pour la prochaine session d'entraînement.
5. **Affichage Markdown :** On utilise Markdown pour afficher les résultats de manière claire et professionnelle, directement dans la sortie du notebook.

In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('..', 'src'))
if module_path not in sys.path:
    sys.path.append(module_path)
    print(f"'{module_path}' ajouté au sys.path")

from tools.dataset_builder import DatasetBuilder

# --- Initialisation du builder ---
# On lui donne les chemins relatifs depuis le dossier du notebook
builder = DatasetBuilder(
    catalog_dir="../data/catalog/",
    raw_data_dir="../data/raw/"
)

# --- Collecte des statistiques ---
# On utilise les méthodes internes de la classe DatasetBuilder
# pour obtenir les chemins des fichiers FITS disponibles et déjà entraînés
all_available_fits_paths = builder._list_available_fits()
already_trained_fits_paths = builder._load_trained_log()

total_available = len(all_available_fits_paths)
total_trained = len(already_trained_fits_paths)
total_new = len([f for f in all_available_fits_paths if f not in already_trained_fits_paths])

# --- Affichage formaté ---
# On utilise du Markdown pour un rendu propre et lisible 
from IPython.display import display, Markdown

md_output = f"""
### Tableau de Bord du Dataset Spectral

---
- **Spectres Totalement Téléchargés :** `{total_available}`
- **Spectres Déjà Utilisés pour l'Entraînement :** `{total_trained}`
---
- **Spectres Nouveaux et Disponibles pour le Prochain Entraînement :** **`{total_new}`**
"""

print("Analyse de l'état du dataset terminée.")
display(Markdown(md_output))

'c:\Users\alexb\Documents\Google_Cloud\alex_labs_google_sprint\astro_spectro_git\src' ajouté au sys.path
Analyse de l'état du dataset terminée.



### Tableau de Bord du Dataset Spectral

---
- **Spectres Totalement Téléchargés :** `9550`
- **Spectres Déjà Utilisés pour l'Entraînement :** `1000`
---
- **Spectres Nouveaux et Disponibles pour le Prochain Entraînement :** **`8550`**
